# Assignment 4 Visione Artificiale,          Rosario Bongiorno

Inizialmente creo dei metodi che mi serviranno per la generazione del dataset, quindi estrarre volti dai video e ritagliare parti di immagini per creare campioni negativi

In [ ]:
from yoloface import face_analysis
import numpy as np
import cv2
from keras.models import model_from_json

In [ ]:
def getsquarebox(box):
    [y,x,w,h]=box
    m=max(w,h)
    xcenter=x+w/2
    ycenter=y+h/2
    newx=xcenter-m/2
    newy=ycenter-m/2
    return [int(newy),int(newx),m,m]


In [ ]:
face=face_analysis()
i=0
for cartella, sottocartelle, files in os.walk(".\Dataset"):
    for file in files:
        if file.endswith(".jpg") and i<=10000:
            #print(cartella)
            image=cv2.imread(cartella+"/"+file)
            img,box,conf=face.face_detection(image_path=cartella+"/"+file,model='tiny')

        try:
            nbox=getsquarebox(box)
            img=frame[nbox[1]+2:nbox[1]+nbox[2]-2,nbox[0]+2:nbox[0]+nbox[2]-2]
            cv2.imwrite('./Rosario/rosario_'+str(i)+'.jpg', img)
            #print(i)
            i=i+1
        except Exception as e:
            pass



In [ ]:
i=0
for cartella, sottocartelle, files in os.walk(".\Sconosciuto"):
    for file in files:
        if file.endswith(".jpg") and i<=10000:
            #print(cartella)
            image=cv2.imread(cartella+"/"+file)
            w,h,_=image.shape

            if ((i % 2)==0):
                #ritaglio l'angolo in alto a destra
                img=image[h-64:h, w-64:w]
            else:
                #ritaglio l'angolo in alto a sinistra
                img=image[0:64, 0:64]
            i=i+1
            cv2.imwrite('./Dataset/Sfondi/sfondo_'+str(i)+'.jpg', img)




In [ ]:
face=face_analysis()
i=0
video = cv2.VideoCapture('./test.mp4')

ret, frame = video.read()
for j in range(500):
        img,box_set,conf=face.face_detection(frame_arr=frame,model='full')

        for box in box_set:
            try:
                nbox=getsquarebox(box)
                img=frame[nbox[1]+2:nbox[1]+nbox[2]-2,nbox[0]+2:nbox[0]+nbox[2]-2]
                cv2.imwrite('./video/test'+str(i)+'.jpg', img)
                i=i+1

            except Exception as e:
                pass

        
        ret, frame = video.read()

Importo le librerie che serviranno e creo il metodo che genera il dataset, data la directory delle immagini

In [ ]:

from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import random
import pickle
import numpy as np
import cv2
import os


In [ ]:

def getclass(str):

    if str=="Rosario":
        return 0
    elif str=="Alessandro":
        return 1
    elif str=="Antonio_S":
        return 2
    elif str=="Daniele":
        return 3
    elif str=="Antonio_B":
        return 4
    elif str=="Sconosciuto":
        return 5
    

def gen_dataset(path):#"./Dataset"

    x_dataset = []
    y_dataset = []


    for cartella, sottocartelle, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                img=cv2.imread(cartella+"/"+file)
                img = cv2.resize(img, (64, 64))
                x_dataset.append(img)
                classe=cartella[10:]
                y_dataset.append(getclass(classe))

    #effettuo uno shuffle del dataset
    ds=list(zip(x_dataset,y_dataset))
    random.shuffle(ds)
    x_dataset,y_dataset=zip(*ds)

    #convero le liste in np array
    x_dataset=np.asarray(x_dataset)
    y_dataset=np.asarray(y_dataset)

    #salvo in locale il dataset con pickle
    file = open('data/x_dataset', 'wb')
    pickle.dump(x_dataset, file)
    file.close()
    file = open('data/y_dataset', 'wb')
    pickle.dump(y_dataset, file)
    file.close()

    return x_dataset, y_dataset

In [178]:
x_dataset, y_dataset=gen_dataset("./Dataset")

In [164]:
file = open('data/x_dataset', 'rb')
x_dataset= pickle.load(file)
file.close()

file = open('data/y_dataset', 'rb')
y_dataset= pickle.load(file)
file.close()

In [180]:
y_dataset.shape

(34167,)

In [181]:
#divido il dataset in trainset e validationset
x_train, x_val, y_train, y_val = train_test_split(x_dataset, y_dataset, test_size=0.20)

#normalizzo i dati
x_train = x_train.astype('float32') / 255. - 0.5
x_val = x_val.astype('float32') / 255. - 0.5

Utilizzo un data generator per creare le coppie che mi serviranno per addestrare la rete siamese, formo coppie positive a cui associo una label 1 e coppie negative a cui associo label 0

In [182]:
def data_gen(x_train, y_train, batch_size):
    while True:
        indici = np.random.randint(0, x_train.shape[0], batch_size)
        sub_set = x_train[indici]
        sub_set_y = y_train[indici]
        pairs = [[a, b] for idx, a in enumerate(sub_set) for b in sub_set[idx + 1:]]
        label = np.asarray([ 1 if a==b else 0 for idx, a in enumerate(sub_set_y) for b in sub_set_y[idx + 1:]])
        x1 = np.asarray([ a[0] for a in pairs])
        x2 = np.asarray([ a[1] for a in pairs])
        yield (x1, x2), label.astype('float32')

In [183]:
data_gen_batch = data_gen(x_train, y_train,32)
data_gen_batch_val = data_gen(x_val, y_val,32)

Definisco adesso la rete neurale

In [184]:
def euclidean_distance(d):
    return K.sum(K.square(d[0] - d[1]), axis=-1)

def aug_layers(shape_input):
    augmentation_layer=tf.keras.Sequential([
        layers.RandomFlip("horizontal",input_shape=shape_input),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.8),
    ])
    return augmentation_layer

# branch convoluzionale che verrà eseguito due volte
def get_branch(shape_input):
    input_img = keras.Input(shape=shape_input)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    y = layers.Flatten()(x)
    model = keras.Model(input_img, y)
    return model

def get_siamese(shape_input=(64,64, 3)):
    branch = get_branch(shape_input)
    input_img_1 = keras.Input(shape=shape_input)
    input_img_2 = keras.Input(shape=shape_input)
    aug_layer_1=aug_layers(shape_input)(input_img_1)
    aug_layer_2=aug_layers(shape_input)(input_img_2)
    des_1 = branch(input_img_1)
    des_2 = branch(input_img_2)
    output = layers.Lambda(euclidean_distance, name='output_layer')([des_1, des_2])
    model = keras.Model([input_img_1, input_img_2] , output)
    model.summary()
    return model


def contrastive_loss(y_true, y_pred, margin = 1):
    return y_true * y_pred + (1.0 - y_true) * tf.math.maximum(margin - y_pred, 0.0)


Addestro la rete

In [ ]:
s = get_siamese()
s.compile(optimizer = "Adam", loss=contrastive_loss,metrics='mse')

callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)

s.fit(
    data_gen_batch,
    epochs=30,
    shuffle=True,
    steps_per_epoch=100,
    validation_data=data_gen_batch_val,
    validation_steps=10,
    callbacks=[callback]
    )

# Salvo il modello addestrato in locale
model_json = s.to_json()
with open("s.json", "w") as json_file:
    json_file.write(model_json)
# Pesi Modello ->> fil h5
s.save_weights("s.h5")
print("Saved model to disk")


In [185]:
# Carichiamo il file json
json_file = open('s1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
a = model_from_json(loaded_model_json)
# Carichiamo i pesi
a.load_weights("s1.h5")
print("Loaded model from disk")

Loaded model from disk


In [186]:
a.compile(optimizer = "Adam", loss=contrastive_loss,metrics='mse')

In [187]:
a.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 model (Functional)             (None, 8192)         14336       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 output_layer (Lambda)          (None,)              0           ['model[0][0]',            

Creo il metodo gen_test_list che mi crea le coppie di immagini che mi servitanno per calcolare il predict, le coppie sono così formate:
-primo elemento l'immagine di cui vogliamo riconoscere il volto
-secondo elemento una immagine random dal dataset
le coppie sono formate per far si che ogni immagine da valutare sia confrontata con almeno una immagine di ogni classe, il parametro n indica quanti confronti faremo per ogni identità.

In [188]:
def gen_test_list(img,x_train,y_train,n):
    img = cv2.resize(img, (64, 64))
    img = img.astype('float32') / 255. - 0.5
    test_list=[]
    for i in range(6):
        ind = np.random.choice(np.where(y_train == i)[0], size=n)
        for j in range(n):
            image=x_train[ind[j]]
            test_list.append((img,image))
    return np.asarray(test_list)



il metodo knn, date le distanze tra le immagini calcolate dalla rete siamese, ritorna la classe dell' elemento più vicino o degli elementi più vicini.
tramite il parametro k si possono scegliere il numero di vicini da considerare

In [189]:
def knn(img,x_train,y_train,k,n):
    list=gen_test_list(img,x_train,y_train,n)
    pred = a.predict((list[:, 0,:, :, :],list[:, 1,:, :, :]))
    #prendo gli indici dei k elementi minori
    indici = np.argpartition(pred, k)[:k]
    print(pred[indici])
    #ottengo la classe corrispondente all'indice
    classi=indici//n
    print(classi)
    #calcolo l'elemento che compare più volte
    conteggio = np.bincount(classi)
    f= np.argmax(conteggio)
    if f==0:
        return 'Rosario'
    elif f==1:
        return 'Alessandro'
    elif f==2:
        return 'Antonio_S'
    elif f==3:
        return 'Daniele'
    elif f==4:
        return 'Antonio_B'
    elif f==5:
        return 'Sconosciuto'






Dopo aver estrapolato alcuni volti dal video di test, genero un test set con il quale poter calcolare l'accuracy

In [190]:
x_test,y_test=gen_dataset("./testset")


In [191]:
def accuracy(y_true, y_pred):
    correct_predictions = np.equal(y_true, y_pred).sum()
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

In [192]:
acc=[]
for k in [1,3]:
    for n in [10, 20, 30]:
        y_pred=[]
        for img in x_test:
            label=knn(img,x_train,y_train,k,n)
            y_pred.append(getclass(label))
        acc.append(accuracy(y_test,y_pred))


2/2 [==============================] - 0s 34ms/step
[0.44817966]
[2]
2/2 [==============================] - 0s 33ms/step
[0.3308639]
[1]
2/2 [==============================] - 0s 33ms/step
[0.39567834]
[1]
2/2 [==============================] - 0s 29ms/step
[0.4787052]
[1]
2/2 [==============================] - 0s 32ms/step
[0.54324144]
[0]
2/2 [==============================] - 0s 31ms/step
[0.36664602]
[2]
2/2 [==============================] - 0s 34ms/step
[0.4473342]
[0]
2/2 [==============================] - 0s 33ms/step
[0.27709824]
[2]
2/2 [==============================] - 0s 30ms/step
[0.43018612]
[1]
2/2 [==============================] - 0s 34ms/step
[0.5305327]
[0]
2/2 [==============================] - 0s 39ms/step
[0.57187104]
[2]
2/2 [==============================] - 0s 35ms/step
[0.39188752]
[1]
2/2 [==============================] - 0s 40ms/step
[0.6276907]
[2]
2/2 [==============================] - 0s 40ms/step
[0.5685557]
[1]
2/2 [==============================] - 0

In [193]:
i=0
for k in [1,3]:
    for n in [10, 20, 30]:
        print('Accuracy con n='+str(n)+' e k='+str(k)+':  '+str(acc[i]))
        i=i+1

Accuracy con n=10 e k=1:  0.2474747474747475
Accuracy con n=20 e k=1:  0.2676767676767677
Accuracy con n=30 e k=1:  0.26262626262626265
Accuracy con n=10 e k=3:  0.24242424242424243
Accuracy con n=20 e k=3:  0.2777777777777778
Accuracy con n=30 e k=3:  0.2474747474747475


Genero il video di testo con le boudary box


In [194]:
from yoloface import face_analysis
import numpy as np
import cv2

face=face_analysis()

# Apri il video
cap = cv2.VideoCapture('./test.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_size = (W, H)

out = cv2.VideoWriter('new_test.mp4',  cv2.VideoWriter_fourcc('M','J','P','G'), fps, frame_size)
# Loop attraverso ogni frame del video

for i in range(2000):

    ret, frame = cap.read()
    _,box_set,conf=face.face_detection(frame_arr=frame, frame_status=True, model='full')
    for box in box_set:
        try:
            nbox=getsquarebox(box)
            img=frame[nbox[1]+2:nbox[1]+nbox[2]-2,nbox[0]+2:nbox[0]+nbox[2]-2]
            label=knn(img,x_train,y_train,3,20)
            cv2.putText(frame, f"{label}", (nbox[0], nbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (nbox[0], nbox[1]), (nbox[0] + nbox[3], nbox[1] + nbox[2]), (0, 255, 0), 2)
        except Exception as e:
            pass

    cv2.imshow('frame',frame)
    out.write(frame)

    # Se viene premuto il tasto 'q' interrompi il loop
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
out.release()
cv2.destroyAllWindows()


yolov3-tiny_face.weights:: status : file already exists
yolov3_tiny_face.cfg:: status : file already exists
face_detection.weights:: status : file already exists
face_detection.cfg:: status : file already exists
4/4 [==============================] - 0s 31ms/step
[0.49436653 0.459856   0.5034876 ]
[1 1 1]
4/4 [==============================] - 0s 32ms/step
[0.51321155 0.6239952  0.76040137]
[0 0 0]
4/4 [==============================] - 0s 31ms/step
[0.45806608 0.2282577  0.48026308]
[1 1 1]
4/4 [==============================] - 0s 34ms/step
[0.7627653  0.72617805 0.752972  ]
[0 1 1]
4/4 [==============================] - 0s 31ms/step
[0.42563602 0.4780937  0.50548315]
[1 1 1]
4/4 [==============================] - 0s 31ms/step
[0.479333  0.489164  0.5507429]
[5 1 1]
4/4 [==============================] - 0s 31ms/step
[0.4137714  0.4965006  0.51474243]
[1 1 1]
4/4 [==============================] - 0s 31ms/step
[0.6156442 0.5767863 0.6733916]
[0 5 1]
4/4 [=============================